# **Introdução ao Conceito de Árvore de Decisão**

Uma árvore de decisão é uma estrutura de modelagem não linear, que simula o processo de decisão com base em respostas a perguntas realizadas sequencialmente aos dados. Cada resposta é interpretada como um nó de divisão no algoritmo, e geometricamente representa uma divisão ortogonal a $n-1$ eixos das variáveis de saída.

Exemplo de uma estrutura de árvore de decisão:

<img src="https://i1.wp.com/www.vooo.pro/insights/wp-content/uploads/2016/12/RDS-Vooo_insights-Tutorial_arvore_de_decisao_02.jpg?resize=640%2C371&ssl=1" width=600>

A próxima imagem apresenta de forma ilustrativa o que acontece com o espaço de atributos.

<img src=https://paulvanderlaken.files.wordpress.com/2020/03/readme-titanic_plot-11.png width=500>

## **Processo de Aprendizado da Árvore de Decisão**

A primeira pergunta que uma árvore precisa responder é: *qual o atributo receberá a divisão?*

Dependendo do atributo que for utilizado no nó raiz, a árvore pode atingir um determinado grau de separação dos conjuntos de cada classe. Então deve existir uma métrica que direciona a separação para um determinado atributo.

<img src="https://i2.wp.com/www.vooo.pro/insights/wp-content/uploads/2016/12/RDS-Vooo-Tutorial_completo_arvore_decisao_03.jpg?resize=617%2C293&ssl=1" width=500>



### *Critério de Gini*

A impureza de Gini mede o quaõ "impuras" são as folhas de uma árvore contruídas após as divisões do nó. É calculada como:

$$Gini(D) = 1- \sum_{i=1}^{n}p_i$$

Essencialmente, a árvode de decisão compara a impureza de Gini antes e depois da realização das divisões em cada atributo, e seleciona aquele atributo que vai proporcionar a **maior purificação** da amostra.

### *Critério de Entropia*

A entropia é uma quantidade definida em física, engenharia e teoria da informação, com o objetivo de quantificar o **grau de desordem** de um sistema ou, de forma equivalente, o quanto se possui de informação a respeito de um sistema. É calculada como:

$$E = -\sum_{i=1}^{n}p_i \log_2{p_i}$$

De mesma forma, a árvore de decisão procura aquele atributo que proporciona a maior queda de entropia, que representa o maior ganho de informação.

In [1]:
import warnings
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split

# ignorar warnings
warnings.filterwarnings('ignore')

In [2]:
# montando drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
# caminho para o arquivo
PATH='/content/drive/MyDrive/Pos_Graduacao/FundamentosMachineLearning/data/german_credit.csv'
data = pd.read_csv(PATH)
data.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [8]:
# modificando os nomes das colunas
data.columns = data.columns.str.lower().str.replace(' ', '_')
data.head()

,creditability,account_balance,duration_of_credit_(month),payment_status_of_previous_credit,purpose,credit_amount,value_savings/stocks,length_of_current_employment,instalment_per_cent,sex_&_marital_status,...,duration_in_current_address,most_valuable_available_asset,age_(years),concurrent_credits,type_of_apartment,no_of_credits_at_this_bank,occupation,no_of_dependents,telephone,foreign_worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [9]:
# separando x e y - vamos usar todos as colunas
x = data.drop(['creditability'], axis=1)
y = data[['creditability']]

In [10]:
# separando treino e teste - com estratificação
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

In [11]:
# treinando nossa primeira árvore de decisão - não precisa escalonamento
dt = DecisionTreeClassifier().fit(x_train, y_train)

# realizando novas previsões
yhat_train = dt.predict(x_train)
yhat_test = dt.predict(x_test)

In [14]:
display(y.value_counts(normalize=True))

,proportion
creditability,
1,0.7
0,0.3


In [12]:
# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       210
           1       1.00      1.00      1.00       490

    accuracy                           1.00       700
   macro avg       1.00      1.00      1.00       700
weighted avg       1.00      1.00      1.00       700



Claramente houve um *overfitting* desse modelo. Ele foi capaz de modelar até os ruídos e variabilidades aleatórias do conjunto de dados, mas não foi capaz de reproduzir essas correlações no conjunto de teste.

Inevitavelmente, se fizermos perguntas o suficiente, vamos conseguir modelar qualquer sistema. No entanto, isso vai tornar o modelo tão complexo e específico, que dificilmente os padrões aprendidos serão reproduzidos em outros conjuntos de dados.

In [13]:
# análise do desempenho
print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.36      0.37      0.36        90
           1       0.73      0.72      0.73       210

    accuracy                           0.62       300
   macro avg       0.54      0.55      0.55       300
weighted avg       0.62      0.62      0.62       300



In [15]:
# Analisando as dimensões da árvore criada
print(f'Profundidade da árvore: {dt.get_depth()}')
print(f'Número de folhas da árvore: {dt.get_n_leaves()}')

Profundidade da árvore: 15
Número de folhas da árvore: 140


In [19]:
from sklearn.tree import export_graphviz
import graphviz

# Create a list of feature names with '&' replaced to avoid Graphviz errors
cleaned_feature_names = [col.replace('&', '_and_') for col in x_train.columns]

# Export the decision tree to a DOT file
dot_data = export_graphviz(dt, out_file=None,
                         feature_names=cleaned_feature_names,
                         class_names=['Good Credit', 'Bad Credit'],
                         filled=True, rounded=True,
                         special_characters=True)

In [20]:
# Render the DOT graph
graph = graphviz.Source(dot_data)
graph.render('decision_tree_plot', view=True, format='png', cleanup=True)

'decision_tree_plot.png'

## **Determinação de Hiperparâmetros por Validação Cruzada**

Um dos desafios de se determinar os valores corretos de hiperparâmetros é que tais valores podem ser dependentes também dos dados que estamos submetendo o modelo. Não necessariamente um conjunto de hiperparâmetros que realizem uma boa previsão num conjunto de dados terá um bom desempenho em outro conjunto de dados.

Assim precisamos buscar um conjunto de hiperparâmetros que apresentem um bom desempenho *médio*, ou seja, que apresentem desempenhos adequados em diversas bases de dados distintas.

Para isso, podemos usar o conceito de **validação cruzada em $k$ folhas**. Veja um exemplo abaixo:

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*AAwIlHM8TpAVe4l2FihNUQ.png">

Na validação cruzada de $k$ folhas, o modelo é treinado $k$ vezes, sendo que em cada vez, uma fração de $1/k$ do conjunto de dados é utilizado como base de teste, enquanto as outras $(k-1)/k$ porções são utilizadas como base de treinamento.

Ao final do treinamento, escolhemos aquele conjunto de dados que resultam no melhor desempenho médio.

E se usarmos uma métrica mais voltada aos negócios para determinar o melhor conjunto de hiperparâmetros?